In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output, display

In [ ]:
LONDON_URL = "https://results.tcslondonmarathon.com/{}/?page={}&event=MAS&num_results=1000&pid=list&pidp=start&search%5Bsex%5D={}&search%5Bage_class%5D=%25"
CHICAGO_URL =  "https://results.chicagomarathon.com/{}/?page={}&event=MAR&num_results=1000&pid=list&pidp=start&search%5Bsex%5D={}&search%5Bage_class%5D=%25"
CHICAGO_22_URL = "https://chicago-history.r.mikatiming.com/{}/?page={}&event=MAR_9TGG9638119&lang=EN_CAP&pid=list&pidp=start&search%5Bsex%5D={}&search%5Bage_class%5D=%25"

In [ ]:
URL = LONDON_URL
YEAR = 2023
NAME = "london_" + str(YEAR)
#SEX = ['M', 'W'] 
SEX = ['M', 'W', 'D'] 


In [ ]:
df_final = None

for sex in SEX:
    page_number = 1

    while True:
        clear_output()
        display(f"Retrieving page {page_number} for sex {sex}")

        res = requests.get(URL.format(YEAR, page_number, sex))
        page = BeautifulSoup(res.text)

        rows = page.find_all("li", class_="row")

        if len(rows) == 2 and rows[1].find("div", class_="alert"):
            break

        rows2 = [r.find_all("div", class_="list-field") for r in rows]

        df = pd.DataFrame([[d.text for d in data] for data in rows2])
        df.insert(0, "Sex", sex)

        if df_final is None:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])

        page_number += 1

df_final.shape

In [ ]:
df_final.head(4)

In [ ]:
if URL==LONDON_URL:
    cols = ['sex', 'place_overall', 'place_gender', 'place_category', 'club', 'bib', 'age', 'xx1', 'half_time', 'time','xx2','xx3']
elif URL==CHICAGO_22_URL:
    cols = ['sex', 'place_overall', 'place_gender', 'bib', 'age', 'time', 'xx']
df_final.columns = cols
df_final.head(3)

In [ ]:

df_final['age'] = df_final['age'].str.replace('Division','')
df_final['age'] = df_final['age'].str.replace('Category','')
df_final['bib'] = df_final['bib'].str.replace('BIB','') 
df_final['bib'] = df_final['bib'].str.replace('Runner Number','') 
df_final['time'] = df_final['time'].str.replace('Finish','')

if 'half_time' in df_final.columns:
    df_final['half_time'] = df_final['half_time'].str.replace('HALF','')
    df_final['half_time'] = df_final['half_time'].str.replace('Half','')
df_final.head(3)

In [ ]:
df_final.reset_index(drop=True, inplace=True)
df_final=df_final[(df_final['age']!='BIBBIB') & (df_final['age']!='Runner NumberRunner Number')]
df_final.reset_index(drop=True, inplace=True)

In [ ]:

df_final.to_csv(NAME + '_full.csv')
df_final[['sex','age','time']].to_csv(NAME + '.csv')

In [ ]:
df_final.pivot_table(index='age', columns='sex', aggfunc='size', fill_value=0)